---
# Database
---

In [234]:
# clone
!git clone https://github.com/sleuoth/ABDA2019.git
!ls ABDA2019/testdaten/cryptominuteresolution
print()

# unzip btcusd
!unzip ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
!mv btcusd.csv ABDA2019/testdaten/cryptominuteresolution/btcusd.csv

fatal: destination path 'ABDA2019' already exists and is not an empty directory.
abseth.csv	cndbtc.csv  foausd.csv	mkrdai.csv  rdnusd.csv	utkbtc.csv
absusd.csv	cndeth.csv  fsnbtc.csv	mkreth.csv  repbtc.csv	utketh.csv
agibtc.csv	cndusd.csv  fsneth.csv	mkrusd.csv  repeth.csv	utkusd.csv
agieth.csv	cnneth.csv  fsnusd.csv	mlneth.csv  repusd.csv	utneth.csv
agiusd.csv	cnnusd.csv  fttusd.csv	mlnusd.csv  reqbtc.csv	utnusd.csv
aidbtc.csv	csxeth.csv  fttust.csv	mnabtc.csv  reqeth.csv	veebtc.csv
aideth.csv	csxusd.csv  funbtc.csv	mnaeth.csv  requsd.csv	veeeth.csv
aidusd.csv	ctxbtc.csv  funeth.csv	mnausd.csv  rifbtc.csv	veeusd.csv
aiobtc.csv	ctxeth.csv  funusd.csv	mtnbtc.csv  rifusd.csv	vetbtc.csv
aioeth.csv	ctxusd.csv  geneth.csv	mtneth.csv  rlcbtc.csv	veteth.csv
aiousd.csv	dadbtc.csv  genusd.csv	mtnusd.csv  rlceth.csv	vetusd.csv
algbtc.csv	dadeth.csv  gnoeth.csv	ncabtc.csv  rlcusd.csv	vldeth.csv
algusd.csv	dadusd.csv  gnousd.csv	ncaeth.csv  rrbusd.csv	vldusd.csv
algust.csv	daibtc.csv  gntbtc.csv	n

In [235]:
!ls ABDA2019/testdaten/cryptominuteresolution -lsS


total 2187184
133308 -rw-r--r-- 1 root root 136506695 Nov  1 22:20 btcusd.csv
 82332 -rw-r--r-- 1 root root  84306728 Dec  6 17:56 ethbtc.csv
 80084 -rw-r--r-- 1 root root  82002667 Dec  6 17:56 ltcusd.csv
 75200 -rw-r--r-- 1 root root  77004739 Dec  6 17:56 ethusd.csv
 71884 -rw-r--r-- 1 root root  73607204 Dec  6 17:56 ltcbtc.csv
 70568 -rw-r--r-- 1 root root  72258961 Dec  6 17:56 xrpusd.csv
 55612 -rw-r--r-- 1 root root  56945273 Dec  6 17:55 eosusd.csv
 54936 -rw-r--r-- 1 root root  56252320 Dec  6 17:56 iotusd.csv
 52676 -rw-r--r-- 1 root root  53937727 Dec  6 17:55 etcusd.csv
 48084 -rw-r--r-- 1 root root  49234953 Dec  6 17:55 etcbtc.csv
 47224 -rw-r--r-- 1 root root  48354612 Dec  6 17:56 xrpbtc.csv
 46796 -rw-r--r-- 1 root root  47915828 Dec  6 17:55 eosbtc.csv
 42484 -rw-r--r-- 1 root root  43500481 Dec  6 17:56 iotbtc.csv
 40156 -rw-r--r-- 1 root root  41116756 Dec  6 17:56 neousd.csv
 39312 -rw-r--r-- 1 root root  40254331 Dec  6 17:55 btceur.csv
 37076 -rw-r--r-- 1 root r

---
# Frameworkinstallation
---

In [236]:
# jdk
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# tree
!apt-get install tree

# spark-package
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tree spark-2.4.4-bin-hadoop2.7

# findspark
!pip install findspark

# numpy
!pip install numpy

# timeseries library
!pip install ts ts-flint #!pip3 install ta-lib --force-reinstall wheel datascience urllib3 kaggle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
The following packages were automatically installed and are no longer required:
  cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-curand-10-1 cuda-curand-dev-10-1
  cuda-cusolver-10-1 cuda-cusolver-dev-10-1 cuda-cusparse-10-1
  cuda-cusparse-dev-10-1 cuda-drivers cuda-license-10-2 cuda-npp-10-1
  cuda-npp-dev-10-1 cuda-nsight-10-1 cuda-nsight-compute-10-1
  cuda-nsight-systems-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1
  cuda-nvjpeg-10-1 cuda-nvjpeg-dev-10-1 cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1
  cuda-nvvp-10-1 default-jre dkms freeglut3 freeglut3-dev
  keyboard-configuration libargon2-0 libcap2 libcryptsetup12 libcublas10
  libdevmapper1.02.1 libfontenc1 libgtk2.0-0 libgtk2.0-common libidn11
  libip4tc0 libjansson4 libnvidia-cfg1-440 libnvidia-common-430
  libnvidia-common-440 libnvidia-decode-440 libnvidia-encode-440
  libnvidia-fbc1-440 libnvidia-gl

---
# Code
*@see [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#formula-and-calculation-for-ppo)*

---

In [237]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

if 'COLAB_TPU_ADDR' not in os.environ:
  print(">> Colab connected to TPU")
else:
  print(">> Colab not connected to TPU")
  
import findspark

import matplotlib.pyplot as plt

findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

#spark = SparkSession.builder.config('spark.driver.extraClassPath', '/path/to/postgresql.jar').getOrCreate()

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

>> Colab connected to TPU


In [0]:
preview_limit = 5  #@param {type: "slider", min: 1, max: 100}
dataframe_from = "Git" #@param ["Git", "Sample: Intel (requieres './intc.csv')"] {allow-input: false}
filename_to_load = 'btcusd.csv'  #@param {type: "string"}



In [248]:
from pyspark.sql.functions import input_file_name

if dataframe_from == "Git":
  df_spark = spark.read.csv('./ABDA2019/testdaten/cryptominuteresolution/'+filename_to_load, inferSchema=True, header=True)
  start = 59
  end = 69

elif dataframe_from == "Sample: Intel (requieres './intc.csv')":
  df_spark = spark.read.csv('./intc.csv', inferSchema=True, header=True)
  start = 17
  end = 26
  # For more info about this example see
  # https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo


df_spark = df_spark \
  .withColumn('filepath', input_file_name()) \
  .withColumn('filename', input_file_name()[start:end]) \
  .withColumn('timestamp', df_spark['time']/1000) \
  .withColumn('date', (df_spark['time']/1000).cast('timestamp'))

print("Type:", type(df_spark), "\n")
print("Schema: ", end = '')
df_spark.printSchema()
df_spark_total_count = df_spark.count()
print("Available rows:", df_spark_total_count, "\n")

df_spark.show(preview_limit)

df_spark.createOrReplaceTempView("base_data")

df_spark_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data group by filename")
df_spark_informationView.show()

Type: <class 'pyspark.sql.dataframe.DataFrame'> 

Schema: root
 |-- time: long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- filepath: string (nullable = false)
 |-- filename: string (nullable = false)
 |-- timestamp: double (nullable = true)
 |-- date: timestamp (nullable = true)

Available rows: 2457265 

+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|         time| open|close| high|  low|     volume|            filepath|  filename|   timestamp|               date|
+-------------+-----+-----+-----+-----+-----------+--------------------+----------+------------+-------------------+
|1364774820000|93.25| 93.3| 93.3|93.25|       93.3|file:///content/A...|btcusd.csv|1.36477482E9|2013-04-01 00:07:00|
|1364774880000|100.0|100.0|100.0|100.0|       93.3|file:///conte

In [249]:
#
# DAY-FILTER
#

base_data_dayfiltered = spark.sql(
  """SELECT d.* FROM base_data d  
     where  (d.date, filename) in (select max(m.date), filename from base_data m group by date_format(m.date, "y-M-d"), filename)
  """)
base_data_dayfiltered.createOrReplaceTempView("base_data_dayfiltered")
#base_data_dayfiltered.show(preview_limit)

base_data_dayfiltered_total_count = base_data_dayfiltered.count()
print("Available rows:", base_data_dayfiltered_total_count, "(lost",(df_spark_total_count-base_data_dayfiltered_total_count), "rows)", "\n")

base_data_dayfiltered_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data_dayfiltered group by filename")
base_data_dayfiltered_informationView.show()

Available rows: 2211 (lost 2455054 rows) 

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



In [0]:
#
# EMA-UDF
#

from pyspark.sql.functions import col, collect_list, concat_ws, udf
from pyspark.sql.types import DoubleType

# see https://tradistats.com/exponentieller-gleitender-durchschnitt/
def ema(ar):
    if len(ar) > 0:
       SF  = 2/ (len(ar)+1)
       SFi = 1 - SF
       my_ema = ar[0]
       for i in ar:
           my_ema = (i * SF) + (my_ema * SFi)
    return my_ema

ema_udf = udf(ema, DoubleType())

In [0]:
#
# EMA-WINDOWS
#

from pyspark.sql.window import Window

win26 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-25, 0))

win12 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-11, 0))

win9 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-8, 0))

In [0]:
#
# EMA_26 && EMA_12 CALCULATION
#

base_data_tmp_ema_1 = base_data_dayfiltered \
  .withColumn('win26_close_list', collect_list('close').over(win26)) \
  .withColumn('win12_close_list', collect_list('close').over(win12))

base_data_tmp_ema_2 = base_data_tmp_ema_1.select(
  "*",
  ema_udf(base_data_tmp_ema_1["win26_close_list"]).alias("EMA26"),
  ema_udf(base_data_tmp_ema_1["win12_close_list"]).alias("EMA12")
)

base_data_tmp_ema_2.createOrReplaceTempView("base_data_tmp_ema_2")
#base_data_tmp_ema_2.show(preview_limit)

In [0]:
#
# PPO CALCULATION
#

base_data_tmp_ema_3 = spark.sql(
  " select *, (((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))   " \
  " FROM base_data_tmp_ema_2 " \
  " order by filename asc, date desc "
)

#
# SIGNAL CALCULATION
#
base_data_tmp_ema_4 = base_data_tmp_ema_3 \
  .withColumn('win9_ema12sub26_list', collect_list('(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))').over(win9))


base_data_tmp_ema_5 = base_data_tmp_ema_4.select(
  "*",
  base_data_tmp_ema_4['(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))'].alias("PPO"),
  ema_udf(base_data_tmp_ema_4["win9_ema12sub26_list"]).alias("SIGNAL")
)

base_data_tmp_ema_5.createOrReplaceTempView("base_data_tmp_ema_5")
#base_data_tmp_ema_5.show(preview_limit)

In [0]:
#
# PPO-HISTOGRAM CALCULATION
#

base_data_tmp_ema_6 = spark.sql(
  " select *, (PPO - SIGNAL)   " \
  " FROM base_data_tmp_ema_5 " \
  " order by filename asc, date desc "
)

base_data_ema = base_data_tmp_ema_6.select(
  "*",
  base_data_tmp_ema_6['(PPO - SIGNAL)'].alias("PPO_HISTOGRAM")
)

base_data_ema.createOrReplaceTempView("base_data_ema")
#base_data_ema.show(preview_limit)

In [255]:
#
# RESULT
#

data_result = spark.sql(
  " select filename, Date(date), format_number(close, "+resultview_accurarcy_str+") close, format_number(EMA26, "+resultview_accurarcy_str+") EMA26, format_number(EMA12, "+resultview_accurarcy_str+") EMA12, format_number(PPO, "+resultview_accurarcy_str+") PPO, format_number(SIGNAL, "+resultview_accurarcy_str+") Signal, format_number(PPO_HISTOGRAM, "+resultview_accurarcy_str+") PPO_Histogram   " \
  " FROM base_data_ema " \
  " order by filename asc, date desc "
)

print("Type:", type(data_result), "\n")
print("Schema: ", end = '')
base_data_ema.printSchema()
print("Available rows:", data_result.count(), "\n")

df_spark_informationView = spark.sql("SELECT filename, count(*), min(date) start_date, max(date) end_date  FROM base_data_ema group by filename")
df_spark_informationView.show()

data_result.show(resultview_limit)

Type: <class 'pyspark.sql.dataframe.DataFrame'> 

Schema: root
 |-- time: long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- filepath: string (nullable = false)
 |-- filename: string (nullable = false)
 |-- timestamp: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- win26_close_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- win12_close_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA26: double (nullable = true)
 |-- EMA12: double (nullable = true)
 |-- (((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE)): double (nullable = true)
 |-- win9_ema12sub26_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- PPO: double (nullable = true)
 |-- SIGNAL: double (nullable = true)
 |-- (PPO - SIGNAL): double (nullable = tr

---
# Analysis *@see [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#what-the-indicator-tells-you)*

---

**→ ppo_trend_analysis** :

Applied to PPO (window of at least 1):

> *When the PPO is above zero, that helps confirm an uptrend since the short-term EMA is above the longer-term EMA. When the PPO is below zero, the short-term EMA is below the longer-term EMA, which is an indication of a downtrend.*


**→ ppo_crossover_signal_analysis** :

Applied to PPO_HISTOGRAM (window of at least 2):
> *The indicator generates a buy signal when the PPO line crosses above the signal line from below, and a sell signal occurs when the PPO line crosses  below the signal from above.*

Applied to PPO (window of at least 2):
> *Centerline crossovers also generate trading signals. Traders consider a move from below to above the centerline as bullish, and a move from above to below the centerline as bearish.*

**→ ppo_technical_divergence_analysis:**

Applied to close and PPO (window of at least 2):
> *Traders can also use the PPO to look for technical divergence between the indicator and price. For example, if the price of an asset makes a higher high, but the indicator makes a lower high, it may indicate the upward momentum is subsiding. Conversely, if an asset's price makes a lower low, but the indicator makes a higher low, it could suggest that the bears are losing their traction and the price could head higher soon.*
---


In [0]:
#
# Analysis-Windows
#

from pyspark.sql.window import Window

win2 = (Window
    .partitionBy("filename")
    .orderBy("date") \
    .rowsBetween(-1, 0))

win3 = (Window
    .partitionBy("filename")
    .orderBy("date")
    .rowsBetween(-2, 0))

In [0]:
#
# Analyse-UDF
#

# see https://www.investopedia.com/terms/p/ppo.asp#what-the-indicator-tells-you

from pyspark.sql.types import StringType

def ppo_trend_analysis(ar):
    output = ""
    count = len(ar)
    if count > 0:
      # comapre the signle-one
      if count == 1:
        current = ar[0]
        if current > 0:
          output = "▲ Aufwärtstrend" # "Aufwärtstrend"
        elif current < 0:
          output = "▼ Abwärtstrend" # "Abwärtstrend"

      # compare the last one, consider change of the penultimate trend
      elif count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1

        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last > 0 and next_to_last > 0:
          output = "▲ Aufwärtstrend" # "Solider Uptrend"
        elif last < 0 and next_to_last > 0:
          output = "↘ C - Abwärtstrend" # "Emerging Downtrend"
        elif last < 0 and next_to_last < 0:
          output = "▼ Abwärtstrend" # "Solid Downtrend"
        elif last > 0 and next_to_last < 0:
          output = "↗ C - Aufwärtstrend" # "Emerging Uptrend"

    return output

def ppo_crossover_signal_analysis(ar):
    output = ""
    count = len(ar)
    # compare last possible crossover
    if count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1
        
        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last <= 0 and next_to_last > 0:
          output = "- Verkaufen" # "Verkauf Signal"
        elif last >= 0 and next_to_last < 0:
          output = "+ Kaufen" # "Kauf Signal"

    return output

def ppo_technical_divergence_analysis(close_ar, ppo_ar):
    output = ""
    count_close = len(close_ar)
    count_ppo = len(ppo_ar)
    
    # ignore if passed arrays do not have the same amount of elements
    if count_close == count_ppo and count_close > 1:
      # calc deltas
      pointer = count_close - 1
      delta_close = close_ar[pointer]
      delta_ppo = ppo_ar[pointer]
      pointer -= 1

      while pointer >= 0:
        delta_close -= close_ar[pointer]
        delta_ppo -= ppo_ar[pointer]
        pointer -= 1

      # calc divergence
      div_close = delta_close / count_close
      div_ppo = delta_ppo / count_ppo

      if div_close > div_ppo:
        output = "↧ Abflachend"
      else:
        output = "↥ Wachsend"

    return output

ppo_trend_analysis_udf = udf(ppo_trend_analysis, StringType())
ppo_crossover_signal_analysis_udf = udf(ppo_crossover_signal_analysis, StringType())
ppo_technical_divergence_analysis_udf = udf(ppo_technical_divergence_analysis, StringType())

In [0]:
#
# SIGNAL ANALYSIS
#

base_data_ema_tmp_eval = base_data_ema \
  .withColumn('win2_ppo_list', collect_list('PPO').over(win2)) \
  .withColumn('win2_ppoh_list', collect_list('PPO_Histogram').over(win2)) \
  .withColumn('win2_close_list', collect_list('close').over(win2))

base_data_ema_eval_result = base_data_ema_tmp_eval.select(
  "*",
  ppo_trend_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("TREND"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("WEAK_SIGNAL"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppoh_list"]).alias("STRONG_SIGNAL"),
  ppo_technical_divergence_analysis_udf(base_data_ema_tmp_eval["win2_close_list"], base_data_ema_tmp_eval["win2_ppo_list"]).alias("DIVERGENCE")
)

base_data_ema_eval_result.createOrReplaceTempView("base_data_ema_eval_result")
#base_data_ema_eval_result.show(preview_limit)

---
# Visualisierung
---

In [259]:
base_data_dayfiltered_informationView.show()

+----------+--------+-------------------+-------------------+
|  filename|count(1)|          min(date)|          max(date)|
+----------+--------+-------------------+-------------------+
|btcusd.csv|    2211|2013-04-01 23:58:00|2019-10-01 18:46:00|
+----------+--------+-------------------+-------------------+



In [0]:
display_limit = 10  #@param {type: "slider", min: 1, max: 10000}
display_accurarcy = 3  #@param {type: "slider", min: 1, max: 10}
template = "plotly_dark" #@param ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none'] {allow-input: false}
startdate = '2018-10-01' #@param {type:"date"}
enddate = '2019-10-01' #@param {type:"date"}

display_accurarcy_str = str(display_accurarcy)

In [0]:
#
# Import Libs
#

import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

In [0]:
#
# Prep Data
#

data_eval_result = spark.sql(
  " select Date(date), high, low, format_number(open, "+display_accurarcy_str+") open, format_number(close, "+display_accurarcy_str+") close" \
  " , format_number(PPO, "+display_accurarcy_str+") PPO, format_number(SIGNAL, "+display_accurarcy_str+") Signal, format_number(PPO_HISTOGRAM, "+display_accurarcy_str+") PPO_HISTOGRAM " \
  " , TREND, WEAK_SIGNAL, STRONG_SIGNAL, DIVERGENCE "\
  " FROM base_data_ema_eval_result " \
  " WHERE date >= '"+ startdate.strip() +" 00:00:00' AND date <= '"+ enddate.strip() +"23:59:59' " \
  " order by filename asc, date asc "
)

data_pd = data_eval_result.toPandas()

data_pd.close = (data_pd['close'].replace('\,','', regex = True).astype(float))
data_pd.open = (data_pd['open'].replace('\,','', regex = True).astype(float))

data_pd.date = pd.to_datetime(data_pd.date)
data_pd.close = pd.to_numeric(data_pd.close)
data_pd.high = pd.to_numeric(data_pd.high)
data_pd.low = pd.to_numeric(data_pd.low)
data_pd.PPO = pd.to_numeric(data_pd.PPO)
data_pd.Signal = pd.to_numeric(data_pd.Signal)
data_pd.PPO_HISTOGRAM = pd.to_numeric(data_pd.PPO_HISTOGRAM)

In [263]:
#
# Charts & Table
#

stock_chart = go.Figure(
    data = [ go.Candlestick( 
        x = data_pd['date'],
        open = data_pd['open'],
        high = data_pd['high'],
        low = data_pd['low'],
        close = data_pd['close']
        )]
)

ppo_chart = go.Figure(
    data = [ go.Scatter(
        x = data_pd['date'],
        y = data_pd['PPO'],
        mode = 'lines+markers',
        name = 'PPO'
    ), go.Scatter(
        x = data_pd['date'],
        y = data_pd['Signal'],
        mode = 'lines+markers',
        name = 'Signal'
    )]
)

ppo_chart.add_trace(
    go.Bar(
        name = 'PPO-Histogram',
        x = data_pd['date'],
        y = data_pd['PPO_HISTOGRAM']
    )
)

stock_chart.update_layout(
    title='Chart für CSV-Datei \'' + filename_to_load + '\'',
    yaxis_title = 'Points',
    template = 'plotly_dark'
)

ppo_chart.update_layout(
    title='PPO-Indikator für CSV-Datei \'' + filename_to_load + '\'',
    yaxis_title = 'Changes in %',
    template = 'plotly_dark'
)

stock_chart.show()
ppo_chart.show()

pd.set_option('display.min_rows', display_limit)

print("")
print("Zeitraum: "+startdate+" → "+enddate)
print()

display(HTML(data_pd.to_html(index=False)))


Zeitraum: 2018-10-01 → 2019-10-01



date,high,low,open,close,PPO,Signal,PPO_HISTOGRAM,TREND,WEAK_SIGNAL,STRONG_SIGNAL,DIVERGENCE
2018-10-01,6601.900000,6600.800000,6600.800,6601.900,0.851,-0.691,1.542,▲ Aufwärtstrend,,,↥ Wachsend
2018-10-02,6538.300000,6527.500000,6538.300,6527.500,1.513,-0.169,1.682,▲ Aufwärtstrend,,,↥ Wachsend
2018-10-03,6503.200000,6503.049072,6503.200,6503.200,1.628,0.171,1.457,▲ Aufwärtstrend,,,↥ Wachsend
2018-10-04,6590.400000,6590.350258,6590.350,6590.400,1.358,0.365,0.992,▲ Aufwärtstrend,,,↧ Abflachend
2018-10-05,6641.500000,6639.900000,6641.500,6639.900,0.898,0.480,0.418,▲ Aufwärtstrend,,,↧ Abflachend
2018-10-06,6595.200000,6592.800000,6592.800,6595.200,0.603,0.473,0.130,▲ Aufwärtstrend,,,↥ Wachsend
2018-10-07,6609.400000,6608.885577,6608.900,6609.400,0.565,0.515,0.050,▲ Aufwärtstrend,,,↧ Abflachend
2018-10-08,6673.300000,6673.200000,6673.300,6673.200,0.787,0.789,-0.001,▲ Aufwärtstrend,,- Verkaufen,↧ Abflachend
2018-10-09,6663.000000,6663.000000,6663.000,6663.000,0.683,0.851,-0.168,▲ Aufwärtstrend,,,↥ Wachsend
2018-10-10,6630.300000,6630.000000,6630.300,6630.074,0.499,0.869,-0.370,▲ Aufwärtstrend,,,↥ Wachsend


---
# Trackable Charts
---

**BTCUSD**: [traidingview.com](https://de.tradingview.com/chart/ecCxGiMv/) *(login required)*

```
+-------------+------------+------------+-------------+-------------+
| TraidingView.com:                                                 |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |    9305.74 |    9384.68 |    10848.14 |    10630.29 |
| EMA12       |    8734.78 |    8810.38 |    11193.74 |    10886.97 |
| PPO         |      -6.14 |      -6.12 |        3.19 |        2.41 |
| SIGNAL      |      -4.56 |      -4.17 |        1.63 |        0.13 |
| PPO_History |      -1.57 |      -1.95 |        1.55 |        2.28 |
+-------------+------------+------------+-------------+-------------+

+-------------+------------+------------+-------------+-------------+
| CALCULATED:                                                       |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |  9327.3017 |  9446.6421 |  10651.5067 |  10673.4093 |
| EMA12       |  8751.5960 |  8842.4748 |  11163.8865 |  10785.1409 |
| PPO         |     -6.172 |    -6.3956 |      4.8104 |      1.0468 |
| SIGNAL      |    -4.4147 |    -4.0178 |      0.9102 |     -1.8162 |
| PPO_History |    -1.7576 |    -2.3778 |      3.9002 |      2.8631 |
+-------------+------------+------------+-------------+-------------+
```



**intc.csv (utf-8)** *--> pretty bad example*

For more info about this example see [School.StockCharts.com](https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo) 

```
time,open,close,high,low,volume
1271203140000,21.16,20.16,20.16,20.16,0.0
1271289540000,20.49,20.49,20.49,20.49,0.0
1271375940000,20.74,20.74,20.74,20.74,0.0
1271462340000,20.77,20.77,20.77,20.77,0.0
1271721540000,20.53,20.53,20.53,20.53,0.0
1271807940000,19.61,19.61,19.61,19.61,0.0
1271894340000,20.02,20.02,20.02,20.02,0.0
1271980740000,19.70,19.70,19.70,19.70,0.0
1272067140000,19.94,19.94,19.94,19.94,0.0
1272326340000,19.62,19.62,19.62,19.62,0.0
1272412740000,19.11,19.11,19.11,19.11,0.0
1272499140000,19.32,19.32,19.32,19.32,0.0
1272585540000,19.61,19.61,19.61,19.61,0.0
1272671940000,19.54,19.54,19.54,19.54,0.0
1272931140000,18.89,18.89,18.89,18.89,0.0
1273017540000,19.33,19.33,19.33,19.33,0.0
1273103940000,19.21,19.21,19.21,19.21,0.0
1273190340000,19.51,19.51,19.51,19.51,0.0
1273276740000,19.55,19.55,19.55,19.55,0.0
1273535940000,19.92,19.92,19.92,19.92,0.0
1273622340000,20.29,20.29,20.29,20.29,0.0
1273708740000,20.58,20.58,20.58,20.58,0.0
1273795140000,20.52,20.52,20.52,20.52,0.0
1273881540000,20.69,20.69,20.69,20.69,0.0
1274140740000,20.67,20.67,20.67,20.67,0.0
1274227140000,20.72,20.72,20.72,20.72,0.0
1274313540000,20.25,20.25,20.25,20.25,0.0
1274399940000,20.56,20.56,20.56,20.56,0.0
1274486340000,20.49,20.49,20.49,20.49,0.0
1274745540000,20.39,20.39,20.39,20.39,0.0

```

